In [1]:
import sys
import os 
sys.path.append('../src/detectron2/projects/DensePose')

In [2]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import pandas as pd

from src.dataset.UFOP import UFOPDataset, labels

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 



In [3]:
ufop_dataset = UFOPDataset()

In [4]:
lbls = labels('../data/LIBRAS-UFOP/labels.txt')

In [5]:
d = dict()
cls = dict() 


for key, values in lbls.items():        
    k = key.split('_')
    person = k[0]
    cat = k[1]
    
    sign_id = int(np.unique(values['labels']))    
        
    if person not in d.keys():
        d[person] = dict()
        
    if cat not in d[person].keys():
        d[person][cat] = 0
        
    d[person][cat] += len(values['labels'])    

        
    if cat not in cls.keys():
        cls[cat] = dict()
        
    if sign_id not in cls[cat].keys():        
        cls[cat][sign_id] = 0   
        
    cls[cat][sign_id] += len(values['labels'])    
    

In [6]:
categories = ['c1', 'c2', 'c3', 'c4']
colors = ['darkgreen', 'darkblue', 'gray', 'cyan']

for category,color in zip(categories, colors):

    plt.figure(figsize=(8,6))

    plt.bar(x=cls[category].keys(), height=cls[category].values(), width=.5, color=color)
    plt.xticks(list(cls[category].keys()))

    for i, v in cls[category].items():
        plt.text(i-0.15, v+0.5, str(v), color='black', fontweight='bold')

    plt.ylim([0, 80])
    plt.ylabel('# samples')
    plt.xlabel('sign id')
    plt.title(category)
    plt.show()

As one may note, for Category 2, there is no class `20`.
Different from the paper's `fig.7`, here the category 2 goes up to class `30` and the whole dataset goes to `57`.
The quantity of the samples matches the paper up to class `19`. After that, the classes seems to be shifted by 1. For example, class `21` here, looks to be class `20` in the paper.

In [7]:
df = pd.DataFrame(d).T
df.sort_index(inplace=True)
df['Total'] = df.sum(axis=1)

tot = pd.DataFrame(df.sum(),).T
df = pd.concat([df, tot])

df

,c1,c2,c3,c4,Total
p1,105,221,87,206,619
p2,103,220,92,204,619
p3,105,234,90,190,619
p4,100,200,83,200,583
p5,99,214,88,199,600
0,512,1089,440,999,3040


This Dataframe corresponds to Table 3 in the papers. It looks ok!

# Training in UFOP

In [32]:
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA, TruncatedSVD


from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer


from src.dataset.UFOP import gen_cv
from src.utils.results import df_results

In [22]:
x_train, y_train = ufop_dataset.load_features(subset='set_1', 
                                              mode='train', 
                                              categ='c2', 
                                              shuffle=True, 
                                              dim=(64,48),
                                              crop_person=True, 
                                              flatten=True
                                             )

x_valid, y_valid = ufop_dataset.load_features(subset='set_1', 
                                              mode='val', 
                                              categ='c2', 
                                              shuffle=True, 
                                              dim=(64,48),
                                              crop_person=True, 
                                              flatten=True
                                             )

In [23]:
cv = gen_cv(y_train, y_valid)

In [24]:
X = np.concatenate((x_train, x_valid))
y = np.concatenate((y_train, y_valid))

print(x_train.shape, x_valid.shape, X.shape)
print(y_train.shape, y_valid.shape, y.shape)

# to save memory
del x_train, y_train, x_valid, y_valid

(707, 3072) (210, 3072) (917, 3072)
(707,) (210,) (917,)


In [29]:
le = LabelEncoder()
y = le.fit_transform(y)

In [27]:
# pipeline class is used as estimator to enable
# search over different model types

base_pipe = Pipeline([   
    ('model', SVC())
])

In [28]:
svc_space = {    
#     'model': Categorical([SVC(C=0.285873, degree=5.0, gamma=0.002535, kernel='linear', )]),
    'model': Categorical([SVC()]),
    'model__C': Real(1e-6, 1e+6, prior='log-uniform'),
    'model__gamma': Real(1e-6, 1e+1, prior='log-uniform'),
    'model__degree': Integer(1,8),
    'model__kernel': Categorical(['linear', 'poly', 'rbf']),
}

In [31]:
opt = BayesSearchCV(
    base_pipe,
    # (parameter space, # of evaluations)
    [(svc_space, 128),],
    cv=cv
).fit(X, y)

/home/wesley.passos/miniconda3/envs/libras/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/home/wesley.passos/miniconda3/envs/libras/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/home/wesley.passos/miniconda3/envs/libras/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/home/wesley.passos/miniconda3/envs/libras/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/home/wesley.passos/miniconda3/envs/libras/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:

In [33]:
df = df_results(opt)
# df_SVD.to_csv('results_UFOP_SVM.csv')
df

,model,model__C,model__degree,model__gamma,model__kernel,mean_test_score,std_test_score,rank
64,"SVC(C=11917.505229671831, degree=2, gamma=0.09...",4.704825,1,10.000000,poly,0.728571,0.0,1
80,"SVC(C=11917.505229671831, degree=2, gamma=0.09...",2.597723,1,10.000000,poly,0.728571,0.0,1
93,"SVC(C=11917.505229671831, degree=2, gamma=0.09...",1055.474358,1,10.000000,poly,0.728571,0.0,1
92,"SVC(C=11917.505229671831, degree=2, gamma=0.09...",1058.502594,1,10.000000,poly,0.728571,0.0,1
91,"SVC(C=11917.505229671831, degree=2, gamma=0.09...",1064.944819,1,10.000000,poly,0.728571,0.0,1
...,...,...,...,...,...,...,...,...
34,"SVC(C=11917.505229671831, degree=2, gamma=0.09...",0.000002,8,10.000000,linear,0.047619,0.0,121
14,"SVC(C=11917.505229671831, degree=2, gamma=0.09...",0.000004,1,0.000002,linear,0.047619,0.0,121
18,"SVC(C=11917.505229671831, degree=2, gamma=0.09...",1000000.000000,1,10.000000,rbf,0.047619,0.0,121
20,"SVC(C=11917.505229671831, degree=2, gamma=0.09...",16.392491,7,4.849443,rbf,0.047619,0.0,121


In [34]:
x_test, y_test = ufop_dataset.load_features(subset='set_1', 
                                              mode='test', 
                                              categ='c2', 
                                              shuffle=True, 
                                              dim=(64,48),
                                              crop_person=True, 
                                              flatten=True
                                             )

y_test = le.transformnsform(y_test)

In [43]:
opt.score(x_test, y_test)

0.7174887892376681

In [63]:
base_pipe = Pipeline([
    ('reduction', TruncatedSVD()),
    ('model', SVC())
])

In [64]:
svc_SVD_space = {
    'reduction': Categorical([TruncatedSVD(random_state=0),]),
    'reduction__n_components': Integer(2, 150),
    'model': Categorical([SVC()]),
    'model__C': Real(1e-6, 1e+6, prior='log-uniform'),
    'model__gamma': Real(1e-6, 1e+1, prior='log-uniform'),
    'model__degree': Integer(1,8),
    'model__kernel': Categorical(['linear', 'poly', 'rbf']),    
}

opt_SVD = BayesSearchCV(
    base_pipe,
    # (parameter space, # of evaluations)
    [(svc_SVD_space, 128),],
    cv=cv
).fit(X, y)

In [65]:
df_SVD = df_results(opt_SVD)
# df_SVD.to_csv('results_UFOP_SVM_SVD.csv')
df_SVD

,model,model__C,model__degree,model__gamma,model__kernel,reduction,reduction__n_components,mean_test_score,std_test_score,rank
4,"SVC(C=82.88070019483594, degree=7, gamma=0.000...",82.880700,7,0.000120,rbf,"TruncatedSVD(n_components=139, random_state=0)",139,0.723810,0.0,1
103,"SVC(C=82.88070019483594, degree=7, gamma=0.000...",56.432082,1,0.000092,rbf,"TruncatedSVD(n_components=139, random_state=0)",150,0.723810,0.0,1
15,"SVC(C=82.88070019483594, degree=7, gamma=0.000...",25.821091,5,0.000145,rbf,"TruncatedSVD(n_components=139, random_state=0)",137,0.719048,0.0,3
120,"SVC(C=82.88070019483594, degree=7, gamma=0.000...",63199.802427,1,6.918645,poly,"TruncatedSVD(n_components=139, random_state=0)",122,0.719048,0.0,3
26,"SVC(C=82.88070019483594, degree=7, gamma=0.000...",56.829740,1,0.000210,rbf,"TruncatedSVD(n_components=139, random_state=0)",126,0.714286,0.0,5
...,...,...,...,...,...,...,...,...,...,...
10,"SVC(C=82.88070019483594, degree=7, gamma=0.000...",75.334118,7,0.000135,poly,"TruncatedSVD(n_components=139, random_state=0)",133,0.047619,0.0,121
9,"SVC(C=82.88070019483594, degree=7, gamma=0.000...",11.765126,4,0.000022,poly,"TruncatedSVD(n_components=139, random_state=0)",106,0.047619,0.0,121
7,"SVC(C=82.88070019483594, degree=7, gamma=0.000...",0.000002,4,0.006998,poly,"TruncatedSVD(n_components=139, random_state=0)",48,0.047619,0.0,121
50,"SVC(C=82.88070019483594, degree=7, gamma=0.000...",54.784777,1,10.000000,rbf,"TruncatedSVD(n_components=139, random_state=0)",138,0.047619,0.0,121


In [60]:
# result on test set
opt_SVD.score(x_test, y_test)

0.7219730941704036

In [ ]:
video = ufop_dataset[30]
frame = video.get_frame(40)
plt.imshow(frame[:,:,::-1])

plt.show()
print(frame.shape)

In [ ]:
# Generate segmentation

In [ ]:
import sys
import os
sys.path.append('../src/detectron2/projects/DensePose/')

from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor


from densepose import add_densepose_config, add_hrnet_config

from densepose.vis.base import CompoundVisualizer
from densepose.vis.bounding_box import ScoredBoundingBoxVisualizer
from densepose.vis.densepose import (
    DensePoseResultsContourVisualizer,
    DensePoseResultsFineSegmentationVisualizer,
    DensePoseResultsUVisualizer,
    DensePoseResultsVVisualizer,
)
from densepose.vis.extractor import CompoundExtractor, create_extractor

In [ ]:
config_file = '../src/detectron2/projects/DensePose/configs/densepose_rcnn_R_50_FPN_s1x.yaml'
model_url = 'https://dl.fbaipublicfiles.com/densepose/densepose_rcnn_R_50_FPN_s1x/165712039/model_final_162be9.pkl'

In [ ]:
VISUALIZERS = {
        "dp_contour": DensePoseResultsContourVisualizer,
        "dp_segm": DensePoseResultsFineSegmentationVisualizer,
        "dp_u": DensePoseResultsUVisualizer,
        "dp_v": DensePoseResultsVVisualizer,
        "bbox": ScoredBoundingBoxVisualizer,
    }

def create_context(vis_specs):        
        visualizers = []
        extractors = []
        for vis_spec in vis_specs:
            vis = VISUALIZERS[vis_spec]()
            visualizers.append(vis)
            extractor = create_extractor(vis)
            extractors.append(extractor)
        visualizer = CompoundVisualizer(visualizers)
        extractor = CompoundExtractor(extractors)
        context = {
            "extractor": extractor,
            "visualizer": visualizer,
#             "out_fname": args.output,
#             "entry_idx": 0,
        }
        return context

In [ ]:
# Inference with a keypoint detection model
cfg = get_cfg()

add_densepose_config(cfg)
add_hrnet_config(cfg)    
cfg.merge_from_file(config_file)

cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7  # set threshold for this model
cfg.MODEL.WEIGHTS = model_url

predictor = DefaultPredictor(cfg)
outputs = predictor(frame)

In [ ]:
context = create_context(('bbox', 'dp_segm',))

visualizer = context["visualizer"]
extractor = context["extractor"]

image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
image = np.tile(image[:, :, np.newaxis], [1, 1, 3])
data = extractor(outputs['instances'])

plt.figure(figsize=(12,8))

image_vis = visualizer.visualize(image, data)
plt.imshow(image_vis)
plt.show()

In [ ]:
ufop_dataset[0].get_movement_frames()

In [ ]:
gei_ = cv2.imread("../data/LIBRAS-UFOP/gei/p1_c2_s12/gei_029_054.png",  cv2.IMREAD_GRAYSCALE)/255

plt.figure()
plt.imshow(gei_, cmap='gray')
plt.show()

In [ ]:
with open("../data/LIBRAS-UFOP/gei/p1_c2_s12/gei_029_054.pkl", 'rb') as f:
    x = pickle.load(f)

In [ ]:
x.shape

In [ ]:
plt.figure()
plt.imshow(x, cmap='gray')
plt.show()